In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
sys.path.append('/root/code')

import h5py as h5
from definitions import LOG_DIR, WEIGHT_DIR, DATASET_DIR
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time
import datetime
import numpy as np
from utils.contrastiveLoss import ContrastiveLoss
from models.SCNN18 import SCNN18
from models.SCNN18Regression import SCNN18Regression
# from models.SCNN18_Sigmoid_oneSecond import SCNN18_Sigmoid
from models.Complex_SCNN18_Sigmoid import Complex_SCNN18_Sigmoid
from models.Complex_SCNN18 import Complex_SCNN18
from models.SCNN18_Sigmoid_First4Modified import SCNN18_Sigmoid_First4Modified
from models.SCNN18_sigmoid_res_21pts import SCNN18_sigmoid_res_21pts
from models.SCNN18_Sigmoid import SCNN18_Sigmoid
from models.SCNN18_Sigmoid_threeSecond import SCNN18_Sigmoid_threeSecond
from models.SCNN18_Target_Task import SCNN18_Target_Task
from models.SCNN18_Sigmoid_Target_Task import SCNN18_Sigmoid_Target_Task
from models.SCNN18_random_aug import SCNN18_random_aug
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import Callback
import utils.dataset as dataset
import logging
from logging import handlers

In [2]:
LOG = logging.getLogger('root')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('SCNN18_0.1second.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')

In [3]:
initLog()

In [4]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [5]:
# Define hyper-parameter
lr = 1.0
max_epochs = 160
batch_size = 100
input_height = 32000
input_width = 1
nb_classes = 10

In [6]:
test_label_range = ['0.5s - 0.6s', '0.6s - 0.7s', '0.7s - 0.8s', '0.8s - 0.9s', '0.9s - 1.0s', 
                    '1.0s - 1.1s', '1.1s - 1.2s', '1.2s - 1.3s', '1.3s - 1.4s', '1.4s - 1.5s']

# test_ds = dataset.load('./SCNN18_0.1second/SCNNR-Jamendo-test.h5')
test_data = dataset.get_dataset_without_label('./SCNN18_0.1second/SCNNR-RWC-test_4.h5')
test_label = dataset.get_ground_truth('./SCNN18_0.1second/SCNNR-RWC-test_4.h5')
test_data = test_data[:]
test_label = test_label[:]

# test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_label[:, 0]))

# test_data = dataset.get_dataset_without_label('./SCNN18_0.1second/SCNN-FMA-nogap-test.h5')
# test_label = dataset.get_ground_truth('./SCNN18_0.1second/SCNN-FMA-nogap-test.h5')
# test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_label[:, 9, :]))

# dataset_length = [i for i, _ in enumerate(test_ds)][-1] + 1
# print(dataset_length)

test_dataset = 'SCNNR-RWC-test_4.h5'
# test_dataset = 'SCNN-RWC-ignoreGap-test.h5'

# test_ds = test_ds.batch(batch_size)
print(test_data.shape)
print(test_label.shape)

(4646, 32000, 1)
(4646, 10)


In [7]:
training_weights = [
    "2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5",
    "2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5",
    "2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5",
    "2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5",
    "2024-06-15_SCNN18_Sigmoid_RWC_training_4_C4.h5"
]

In [8]:
strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])

In [9]:
def load_model_with_weights(input_shape, nb_classes, weight_file):
    with strategy.scope():
        model = SCNN18_Sigmoid(input_shape, nb_classes).model()
        optimizer = get_optimizer('adadelta', lr)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        model.load_weights(os.path.join(WEIGHT_DIR, weight_file))
    return model

In [10]:
input_shape = (input_height, input_width)
models = [load_model_with_weights(input_shape, nb_classes, weight) for weight in training_weights]

In [11]:
# 使用 N 個分類器進行多數決
# predictions = [model.predict(test_data) for model in models]
# predictions = np.array(predictions)

# for i in range(nb_classes):
#     predict_true = 0

#     for j in range(predictions.shape[1]):
#         vocal_num = 0
#         non_vocal_num = 0

#         for k in range(predictions.shape[0]):
#             if(predictions[k, j, i] < 0.5):
#                 non_vocal_num += 1
#             else:
#                 vocal_num += 1
            
#         if(vocal_num > non_vocal_num and test_label[j, i] == 1):
#             predict_true += 1
#         elif(vocal_num < non_vocal_num and test_label[j, i] == 0):
#             predict_true += 1

#     acc = predict_true/predictions.shape[1]

#     LOG.info(f'loaded_weight={training_weights}, test_dataset={test_dataset}, second_range={test_label_range[i]}, acc={acc:.6f}')

In [12]:
# 使用 N 個分類器進行平均數後以 0.5 做閾值
predictions = [model.predict(test_data) for model in models]
predictions = np.array(predictions)
avg_predictions = np.mean(predictions, axis=0)
print(avg_predictions.shape)

for i in range(nb_classes):
    predict_true = 0

    for index, result in enumerate(avg_predictions):
        if result[i] < 0.5 and test_label[index][i] == 0:
            predict_true += 1
        elif result[i] >= 0.5 and test_label[index][i] == 1:
            predict_true += 1

    acc = predict_true/avg_predictions.shape[0]

    LOG.info(f'loaded_weight={training_weights}, test_dataset={test_dataset}, second_range={test_label_range[i]}, acc={acc:.6f}')

2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4_C4.h5'], test_dataset=SCNNR-RWC-test_4.h5, second_range=0.5s - 0.6s, acc=0.909384
2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4_C4.h5'], test_dataset=SCNNR-RWC-test_4.h5, second_range=0.6s - 0.7s, acc=0.911967
2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4

(4646, 10)


2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4_C4.h5'], test_dataset=SCNNR-RWC-test_4.h5, second_range=1.1s - 1.2s, acc=0.912613
2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4_C4.h5'], test_dataset=SCNNR-RWC-test_4.h5, second_range=1.2s - 1.3s, acc=0.910891
2024-06-18 03:48 INFO loaded_weight=['2024-06-14_SCNN18_Sigmoid_RWC_training_4_C0.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C1.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C2.h5', '2024-06-14_SCNN18_Sigmoid_RWC_training_4_C3.h5', '2024-06-15_SCNN18_Sigmoid_RWC_training_4

In [13]:
# results = model.predict(test_data)
# print(results.shape)

# for i in range(1, 10):
#     threshold = i * 0.1
#     accs = []
#     for j in range(nb_classes):
#         predict_true = 0

#         for index, result in enumerate(results):
#             if result[j] < threshold and test_label[index][j] == 0:
#                 predict_true += 1
#             elif result[j] >= threshold and test_label[index][j] == 1:
#                 predict_true += 1

#         acc = predict_true/results.shape[0]
#         accs.append(acc)

#     LOG.info(f'threshold = {threshold:.1f}, loaded_weight={training_weight}, test_dataset={test_dataset}, acc={np.round(np.mean(accs)*100, 4)}%')

In [14]:
# complex output using softmax

# predictions = model.predict(test_data)
# results = np.square(predictions[0]) + np.square(predictions[1])

# predict_true = 0
# data_size = results.shape[0]
# print(data_size)

# for index, result in enumerate(results):
#     if result[0] > result[1] and test_label[index][1] == 0:
#         predict_true += 1
#     elif result[1] > result[0] and test_label[index][1] == 1:
#         predict_true += 1
#     elif result[1] == result[0]:
#         data_size -= 1

# acc = predict_true/data_size

# LOG.info(f'loaded_weight={training_weight}, test_dataset={test_dataset}, acc={acc:.6f}')

In [15]:
# determinate threshold for complex sigmoid

# test_data_for_threshold = test_data[:int(len(test_data)/10)]
# test_label_for_threshold = test_label[:int(len(test_label)/10)]

# predictions = model.predict(test_data_for_threshold)
# results = np.sqrt(np.square(predictions[0]) + np.square(predictions[1]))

# accs = []
# data_size = results.shape[0]
# print(data_size)

# for i in range(0, int(np.sqrt(2)*1000)):
#     predict_true = 0
#     t = i / 1000
#     for index, result in enumerate(results):
#         if result <= t and test_label_for_threshold[index] == 0:
#             predict_true += 1
#         elif result > t and test_label_for_threshold[index] == 1:
#             predict_true += 1

#     acc = predict_true/data_size
#     accs.append(acc)

# threshold = np.argmax(np.array(accs))/1000
# predictions = model.predict(test_data)
# results = np.sqrt(np.square(predictions[0]) + np.square(predictions[1]))
# predict_true = 0
# data_size = results.shape[0]
# print(data_size)

# for index, result in enumerate(results):
#     if result <= threshold and test_label[index] == 0:
#         predict_true += 1
#     elif result > threshold and test_label[index] == 1:
#         predict_true += 1

# acc = predict_true/data_size

# LOG.info(f'loaded_weight={training_weight}, test_dataset={test_dataset}, acc={acc:.6f}')